In [ ]:
import os
import re
import pickle
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, Embedding, Bidirectional,LSTM, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from  keras . utils  import  to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D,Conv2D, MaxPooling1D, Embedding, Flatten
from keras.models import Model
from keras.initializers import Constant
from keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping

#from fast_text import FastText


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd 
df_train=pd.read_csv('gdrive/My Drive/Thesis Data/full doc csv/full_doc_train.csv')
df_test=pd.read_csv('gdrive/My Drive/Thesis Data/full doc csv/full_doc_test.csv')
total_data = pd.concat([df_train,df_test])
# we have to works with word level,if we works with charchter level then it exits the RAM limit
# here remove 'politics','education','sports' labels of text which is frequently occur in dataset and set it to news_3less.csv file
total_data = total_data[total_data['label']!='politics'] # remove the label which is 'politics'
total_data = total_data[total_data['label']!='education'] # remove the label which is 'education'
total_data = total_data[total_data['label']!='sports'] # remove the label which is 'sports'
print(len(total_data))
# df.to_csv(news/'news_3less.csv',index=False) # convert the data to csv format

51690


In [ ]:
set(total_data.label)

{'accident',
 'art',
 'crime',
 'economics',
 'entertainment',
 'environment',
 'international',
 'opinion',
 'science_tech'}

In [ ]:
!git clone -l -s https://github.com/banglakit/bengali-stemmer.git

Cloning into 'bengali-stemmer'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 94 (delta 5), reused 16 (delta 4), pack-reused 69
Unpacking objects: 100% (94/94), done.


In [ ]:
!pip install git+https://github.com/banglakit/bengali-stemmer.git    

  Cloning https://github.com/banglakit/bengali-stemmer.git to /tmp/pip-req-build-roqrk031
  Running command git clone -q https://github.com/banglakit/bengali-stemmer.git /tmp/pip-req-build-roqrk031
  Created wheel for bengali-stemmer: filename=bengali_stemmer-0.0.1-py2.py3-none-any.whl size=6393 sha256=ef0daaa3ca9e738ac7d2a6e9f3de895b055a5ea51241a44fd7abc13ed9508c4b
  Stored in directory: /tmp/pip-ephem-wheel-cache-uousxocq/wheels/a1/ad/a1/4ba354059b17c00600a14e13a504e7bdb49f20f2f4e2f3639c
Successfully built bengali-stemmer


In [ ]:
!pip install git+https://github.com/banglakit/bengali-stemmer.git    

  Cloning https://github.com/banglakit/bengali-stemmer.git to /tmp/pip-req-build-15n5pe1v
  Running command git clone -q https://github.com/banglakit/bengali-stemmer.git /tmp/pip-req-build-15n5pe1v
  Created wheel for bengali-stemmer: filename=bengali_stemmer-0.0.1-py2.py3-none-any.whl size=6393 sha256=44d9db7d3b4d300223b63e2b1234c666497be5b29fa2c6ef6c604fe85aabd872
  Stored in directory: /tmp/pip-ephem-wheel-cache-t8tefp92/wheels/a1/ad/a1/4ba354059b17c00600a14e13a504e7bdb49f20f2f4e2f3639c
Successfully built bengali-stemmer


In [ ]:
from bengali_stemmer.rafikamal2014 import RafiStemmer
stemmer = RafiStemmer()
stemmer.stem_word('বাংলায়')

'বাংলা'

In [ ]:
# total_data = df_train
# total_data=total_data.append(df_test, ignore_index = True)

In [ ]:
macronum=sorted(set(total_data['label']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

total_data['label']=total_data['label'].apply(fun)


In [ ]:
texts = list(total_data['text'])
labels = list(total_data['label'])


In [ ]:
def clean_punct(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#|।|’|‘]', r'', sentence)
    cleaned1 = re.sub(r'[.|,|(|)|\|/]', r'', cleaned)
    cleaned = re.sub(r'[০|১|২|৩|৪|৫|৬|৭|৮|৯]', r'', cleaned1)
    cleaned1 = re.sub(r'[-|=]', r' ', cleaned)
    return cleaned1

In [ ]:
stopwords = pd.read_csv('gdrive/My Drive/Colab Notebooks/Stopwords.csv')

In [ ]:
set_stop = set(stopwords['words'])

In [ ]:

def pre_process(data):
    i=0
    str1=' '
    final_string = []
    final_words = []
    all_negative_words = []
    s=''

    for sentence in data:
        filtered_sentence = []

        for w in sentence.split():
            for cleaned_word in clean_punct(w).split():
                if len(cleaned_word)>2:
                    if((cleaned_word) not in set_stop):
                        s = stemmer.stem_word(cleaned_word)
                        if len(s)>2:
                            final_words.append(s)
                            filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue

        str1 = " ".join(filtered_sentence)
        final_string.append(str1)
    return final_string

In [ ]:
texts = pre_process(texts)

In [ ]:
texts[0]

'উচ্চ মাধ্যমিক পরীক্ষ মঙ্গলব বিবৃতি মন্ত্রী “আমি হরতাল অবরোধ সন্ত্রাসী কার্যক্রম পরিচালনাকারী জোট আহ্বান জানা পরীক্ষার্থী শান্তিপূর্ণভাব পরীক্ষা দয়া হটকারী ঘটনা ঘটাব “আমি ভাষা দেশ স্থান একজন পরীক্ষার্থীর ক্ষতি দায়দায়িত্ব আপনা বহন মানুষ আপনা ক্ষমা না” জানুয়ারি দেশ টানা অবরোধ চালি আসা বিএনপি জোট ফেব্রুয়ারি মার্চ বেশিরভাগ প্রতিদিন হরতাল বিএনপি জোট অবরোধ হরতাল চলতি এসএসসি সমমান অর্থাৎ দিন পরীক্ষা পেছা বাধ্য শিক্ষা মন্ত্রণাল পরীক্ষা দিন শুক্র শনিব ধরন রাজনৈতিক কর্মসূচির কারণ এইচএসসি পরীক্ষা পেছানো রেখ শিক্ষামন্ত্রী ফাইল বিবৃতি নাহিদ “সংকট এসএসসি পরীক্ষা শেষ এইচএসসি পরীক্ষা জাতির দুর্ভাগ্য রাজনৈতিক জোট বিবেকবর্জিত অব্যাহত হরতাল অবরোধ কারণ এসএসসির রুটিন পরীক্ষা সম্ভব “পরীক্ষার্থী অভিভাবক শিক্ষকসহ শ্রেণি পেশ মানুষ মতামত পরামর্শ সম্মান দেখি পরিস্থিতি রুটিনমাফিক পরীক্ষা সিদ্ধান্ত গ্রহণ করে ব্যত্য না” পরীক্ষার্থী যাতায়াত নির্বিঘ্ন আইনশৃঙ্খলা রক্ষাবাহিনীর সর্বাত্মক নজরদারি আশ্বস্ত শিক্ষামন্ত্রী “ইতিবাচক দৃষ্টিভঙ্গিসম্পন্ন রাজনৈতিক নেতা কর্মী সামাজিক সাংস্কৃতিক সংগঠন সদস্য জনগণ তোমা পাশ তোম নিশ্চিন

In [ ]:
def load_data(num_words, sequence_length, test_size=0.25, oov_token=None):
    tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
    num_classes = 9 
    y = to_categorical(np.asarray(labels), num_classes)
    tokenizer.fit_on_texts(texts)
    X = tokenizer.texts_to_sequences(texts)
    X = np.array(X)
    # pad sequences with 0's
    X = pad_sequences(X, maxlen=sequence_length)
    # split data to training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1)
    data = {}
    data["X_train"] = X_train
    data["X_test"]= X_test
    data["y_train"] = y_train
    data["y_test"] = y_test
    data["tokenizer"] = tokenizer
    return data

In [ ]:
# train_data=pd.read_csv('gdrive/My Drive/Colab Notebooks/ulm_train.csv')
import numpy as np

def get_embedding_vectors(word_index, embedding_size=100):
    
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_size))
    with open(f"gdrive/My Drive/Colab Notebooks/bn_glove.{embedding_size}d.txt", encoding="utf8") as f:
        for line in tqdm(f, "Reading GloVe"):
            values = line.split()
            # get the word as the first word in the line
            word = values[0]
            if word in word_index:
                idx = word_index[word]
                # get the vectors as the remaining values in the line
                embedding_matrix[idx] = np.array(values[1:], dtype="float32")
    return embedding_matrix

In [ ]:
d = set()
for s in texts:
    for ss in list(s.split()):
        d.add(ss)

In [ ]:
len(d) # here is the total number of words in corpus, but the number of tokenizer is small then that(ther are some stopwords which is removed)

278693

In [ ]:
max([len(s.split()) for s in texts]) # find the max length

7779

In [ ]:
len(data['tokenizer'].word_index)

NameError: ignored

In [ ]:
MAX_SEQUENCE_LENGTH = 7779    #max([len(s.split()) for s in texts]) 
MAX_NUM_WORDS = 272611 + 1 # you have to use word same as tokenizer length, and add extra 1(for 0th index case)
vocab_size = MAX_NUM_WORDS
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

data = load_data(MAX_NUM_WORDS , MAX_SEQUENCE_LENGTH,VALIDATION_SPLIT)

In [ ]:
len(data['tokenizer'].word_index) # after running previous cell of code we run this cell,we call the 'load_data' function before we see the len of data['tokenizer']

272611

In [ ]:
embedding_matrix = get_embedding_vectors( data['tokenizer'].word_index ,EMBEDDING_DIM )

Reading GloVe: 134256it [00:10, 13032.93it/s]


In [ ]:
# create the model
embedding_vecor_length = 300

model = Sequential()

model.add(Embedding(MAX_NUM_WORDS, embedding_vecor_length,  weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH))

model.add(GlobalAveragePooling1D(name='gapl')) # Will average the embeddings of all words in the document

model.add(Dense(9, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7779, 300)         81783600  
_________________________________________________________________
gapl (GlobalAveragePooling1D (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 9)                 2709      
Total params: 81,786,309
Trainable params: 81,786,309
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
'''from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

from text_birnn import TextBiRNN

print('Build model...')
# model = TextBiRNN(maxlen, max_features, embedding_dims)
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)
'''

In [ ]:
print('Train...')

history = model.fit(data['X_train'], data['y_train'],batch_size=32, validation_data=(data['X_test'], data['y_test']), epochs=10, verbose=2)

Train...
Epoch 1/10
1293/1293 - 1094s - loss: 1.9980 - accuracy: 0.2914 - val_loss: 1.8301 - val_accuracy: 0.4158
Epoch 2/10
1293/1293 - 1101s - loss: 1.6197 - accuracy: 0.4719 - val_loss: 1.4432 - val_accuracy: 0.5653
Epoch 3/10
1293/1293 - 1106s - loss: 1.2724 - accuracy: 0.6370 - val_loss: 1.1718 - val_accuracy: 0.7027
Epoch 4/10
1293/1293 - 1105s - loss: 1.0395 - accuracy: 0.7311 - val_loss: 1.0031 - val_accuracy: 0.7600
Epoch 5/10
1293/1293 - 1105s - loss: 0.8868 - accuracy: 0.7755 - val_loss: 0.9091 - val_accuracy: 0.7657
Epoch 6/10
1293/1293 - 1103s - loss: 0.7806 - accuracy: 0.8040 - val_loss: 0.8193 - val_accuracy: 0.7973
Epoch 7/10
1293/1293 - 1102s - loss: 0.6977 - accuracy: 0.8261 - val_loss: 0.7729 - val_accuracy: 0.7987
Epoch 8/10
1293/1293 - 1106s - loss: 0.6318 - accuracy: 0.8415 - val_loss: 0.7375 - val_accuracy: 0.8106
Epoch 9/10
1293/1293 - 1108s - loss: 0.5740 - accuracy: 0.8560 - val_loss: 0.6913 - val_accuracy: 0.8240
Epoch 10/10
1293/1293 - 1105s - loss: 0.5230 -

In [ ]:
history = model.fit(data['X_train'], data['y_train'],batch_size=32, validation_data=(data['X_test'], data['y_test']), epochs=5, verbose=2)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
def confussion_mat(model):
    # Predict the values from the validation dataset
    Y_pred = model.predict(data['X_test'])
    # Convert predictions classes to one hot vectors 
    Y_pred_classes = np.argmax(Y_pred,axis = 1) 
    # Convert validation observations to one hot vectors
    Y_true = np.argmax(data['y_test'],axis = 1) 
    # compute the confusion matrix
    confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
    # plot the confusion matrix
    f,ax = plt.subplots(figsize=(15, 15))
    sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.show()

In [ ]:
def history2(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
history2(history)

In [ ]:
confussion_mat(model)